## 实验分析
### 实验内容
这个实验主要内容是,对比直接召回,简单NER,人工擦除,Entity_Agonstic方法的差距
- 直接召回, 使用原始的query来召回数据
- 简单NER, 使用hanlp对query进行擦除后,使用擦除后的query来召回数据
- 人工擦除, 人工对query进行擦除后,使用擦除后的query来召回数据(实际上要素改写+hanlp+re替换)
- Entity_Agonstic方法,利用我们的方法来做embedding, 然后使用embedding来召回数据

这里我个人觉得甚至可以再加一个使用大模型来擦除query的方法

### 数据以及模型的准备

这里针对的数据我个人觉得应该还是question_judgement == Correct的数据
对所有数据都需要其原始query,简单ner之后的query_ner, 人工擦除之后的query_erased

针对前三个方法,使用对应的query和qwen3-4b-embedding作为embedding模型,随后召回
这里的embedding模型,我们可以多换几个,例如qwen3-0.6B, qwen3-8B, 和一些经典的sentence_transformer模型

对于我们的方法,直接使用我们训练好的模型对query进行embedding, 然后使用embedding来召回数据

### 模板库的构建
这里构建模板库,是个需要商榷的问题,是使用之前的方式来吗,相当于是使用人工擦除的方法来做模板,这里需要商讨,先暂时假定为模板库已经生成完毕



### 模块功能分析
*模块一,数据处理*
根据不同的方法,调用不同的函数来处理数据,比如人工擦除,大模型擦除,NER擦除等
这块是重点，需要加入之前的rewriter功能，或者说可以处理好数据指定一下目标列？√


*模块二,模板召回*
这里需要两个功能,一个是embedding功能根据不同的方法调用不同的模型,一个是相似度召回(现成)


*模块三,配置生成*
现成的功能,原始问题+模板prompt+模型调用


*模块四,过程pipeline*
整个过程的控制
pipeline
- 设置模板
- 设置数据处理方法
- 设置embedding模型
- 


## 路径确保

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
import os
# 获取当前notebook文件的目录
current_dir = os.path.dirname(os.path.abspath('.'))
sys.path.append(current_dir)
os.chdir(current_dir)

## 实验

In [ ]:
from multithread import run_all_experiments,create_custom_experiments
experiments = create_custom_experiments(test_path="./data/20250916/trading_ex_template_0.95_4.json",
                                        example_path="./data/20250916/trading_template_0.95_4.json")
run_all_experiments(experiments)

In [ ]:
from hybrid_rewrite.rewrite import evaluate

class RecallPipeline:
    def __init__(self):
        pass

    def run(self,args):
        evaluate(
            json_path=args.get('dataset', None),
            test_path=args.get('test_path', None),
            example_path=args.get('example_path', None),
            prompt_version=args.get('prompt', None),
            split_ratio=args.get('split_ratio', 0.9),
            split_keywords=args.get('split_keywords', False),
            keywords_for_dimvals=args.get('keywords_for_dimvals', True),
            pattern_key=args.get('pattern_key', 'query'),
            pattern_top_k=args.get('pattern_top_k', 10),
            keyword_key=args.get('keyword_key', 'erased_dimension_value'),
            keyword_top_k=args.get('keyword_top_k', 5),
            dim_top_k=args.get('dim_top_k', 1),
            model=args.get('model', None),
            emb_model=args.get('emb_model', None),
            emb_model_path=args.get('emb_model_path', None),
            temperature=args.get('temperature', 0.1),
            merge_retrieval=args.get('merge_retrieval', False)
        )

In [ ]:
config = {
    "dataset": None,
    "test_path": "./data/20250916/commerce_test_0.95_4.json",
    "example_path": "./data/20250916/commerce_template_0.95_4.json",
    "prompt": "0_0_8",
    "split_ratio": 0.9,
    "split_keywords": False,
    "keywords_for_dimvals": True,
    "pattern_key": "query",  # 核心
    "pattern_top_k": 5,
    "keyword_key": "erased_dimension_value",
    "keyword_top_k": 20,
    "dim_top_k": 2,
    "model": "qwen2.5-72b-instruct",            # 核心，改写主模型
    "emb_model": "qwen3-embedding-4b",
    "emb_model_path": None,
    "temperature": 0.1,
    "merge_retrieval": False
}

In [ ]:
# ["qwen3-embedding-0.6b", "qwen3-embedding-4b", "contrastive-bert", "teacher-student"]

# pipe = RecallPipeline()

# pipe.run(config)